# **FolT Homework 10**


### Homework 10.1 (10 points)
For the sake of science, let’s assume that we are on the other side as a spammer.

(a) What could you do to avoid your spam being detected with respect to the previously implemented features?

- Formulate at least two counter-strategies. (2 points)

Keep in mind that as a spammer all you need is the short attention of
the reader for the main message. It does not really matter what the rest of the email looks like. Also the spam
message only needs to be readable, not unaltered. Please take care, that you only change the spam messages
(the messages from the corpus in the training and the test data that belong to the SPAM class), not the valid
emails, as (hopefully) the spammer has no way of changing the valid emails on the user’s computer


__Answer:__

- Removes features that is famous for spam such as repeated characters, in general we want to clean the text

- Add good word that are often associate with good email 

(b) Implement your ideas by adding a method that changes the content of spam emails before they are feed into
the classifier for training.

- write a function to read the mails from the ```emails.zip```. You may use the ```LazyCorpusLoader``` (1 point)



In [5]:
import nltk
from nltk.corpus.util import LazyCorpusLoader
from nltk.corpus.reader import CategorizedPlaintextCorpusReader 

mails_true = LazyCorpusLoader('mails', CategorizedPlaintextCorpusReader,r'(?!\.).*\.txt', cat_pattern=r'(spam|nospam)/.*')

print(mails_true.categories()) #['nospam', 'spam']
print(len(mails_true.fileids('spam'))) #481
print(len(mails_true.fileids('nospam'))) #2412

['nospam', 'spam']
481
2412


- implement at least two of your ideas from (a) (4 points)

In [27]:
import re

#1. First approach: remove special character and repeated character
def clean_txt(text):
    text = re.sub(r"[^A-Za-z ]+", ' ',text) 
    text = text.replace('\n', ' ').replace('\t', ' ')
    text = re.sub(r"[^A-Za-z ]+", '',text) 
    text = [tok for tok in text.split() if len(tok)>1 ]
    text = " ".join(text)
    text = " ".join(text.split())
    
    return text 

In [ ]:
# Get the name of all spam *.txt file

spam_txt =[]
for root, dirs, files in os.walk("emails\spam"):
    for file in files:
        if file.endswith(".txt"):
             spam_txt.append(os.path.join(root, file))

for path in spam_txt:
    with open(path,"r") as f: 
        text = f.read()
        text_modify = clean_txt(text)
    
    filename = "emails/spam1/"+ path.split("\\")[-1]
    with open(filename,"w") as f:
        f.write(text_modify)

In [48]:
from nltk.corpus import words
from random import sample

# 2. Approach add a number of random words in the vocabulary 
def add_valid_word(text, num = 300):
    rand_words = ' '.join(sample(words.words(), num))
    text = text + rand_words
    return text

In [50]:
import os
# Get the name of all spam *.txt file

spam_txt =[]
for root, dirs, files in os.walk("emails\spam"):
    for file in files:
        if file.endswith(".txt"):
             spam_txt.append(os.path.join(root, file))

for path in spam_txt:
    with open(path,"r") as f: 
        text = f.read()
        text = clean_txt(text)
        text_modify = add_valid_word(text)
    
    filename = "emails/spam2/"+ path.split("\\")[-1]
    with open(filename,"w") as f:
        f.write(text_modify)

- Use the classifier that you implemented in the Exercise 10. (1 point)


In [1]:
def compute_PRF(gold, predicted, class_label):
    if len(gold) != len(predicted):
        raise ValueError('Sizes of gold standard and predicted value need to be equal.')

    TP = 0
    FP = 0
    FN = 0
    for i in range(len(gold)):
        if gold[i] == class_label and predicted[i] == class_label:
            TP += 1
        elif not gold[i] == class_label and predicted[i] == class_label:
            FP += 1
        elif gold[i] == class_label and not predicted[i] == class_label:
            FN += 1

    if (TP+FP > 0):
        precision = TP/(TP+FP)
    else:
        precision = 0

    if (TP+FN > 0):
        recall = TP/(TP+FN)
    else:
        recall = 0

    if precision > 0 and recall > 0:
        fmeasure = 2 * precision * recall / (precision + recall)
    else:
        fmeasure = 0

    return (precision, recall, fmeasure)


In [2]:
import nltk, random, logging, math
from corpus_mails import mails
logging.basicConfig(level=logging.DEBUG)

def get_feature_set(mails, fileid, top_words, brown_words):
    fd = nltk.FreqDist(token for token in mails.words(fileid) if token.lower() in top_words)
    features = {}
     
    # use one percent as a threshold - could also be tuned
    one_percent = math.ceil(fd.N() / 100)

    bigrams = nltk.bigrams(mails.words(fileid))

    features['many_exclamation_marks'] = fd['!'] > one_percent
    features['multiple_exclamation_marks'] = len([(w1.lower(), w2.lower()) for (w1, w2) in bigrams if w1 == "!" and w2 == "!"]) > 0
    features['many_stars'] = fd['*'] > one_percent
    features['many_dollars'] = fd['$'] > one_percent
    features['multiple_dollars'] = len([(w1.lower(), w2.lower()) for (w1,w2) in bigrams if w1 == "$" and w2 == "$"]) > 0
    features['many_minus'] = fd['-'] > one_percent
    features['contains_links'] = len([ word for word in fd.keys() if word. startswith('http')]) > 0
    features['many_repetitions'] = len([(w1.lower(), w2.lower()) for (w1,w2) in bigrams if w1 == w2 and w1 in top_words and w2 in top_words]) > one_percent
    features['many_out_of_vocab'] = len([w for w in fd.keys() if not w in brown_words]) > one_percent

    for token in fd.keys():
        features[token] = token

    return features

In [3]:
def evaluate(mail, train_set, test_set, show_features, class_label, top_words, brown_words):

    train_features = [(get_feature_set(mail,fileid,top_words,brown_words), category) for (fileid, category) in
    train_set]

    classifier = nltk.NaiveBayesClassifier.train(train_features)
    if show_features:
        classifier.show_most_informative_features(20)

    results = classifier.classify_many([get_feature_set(mail,fileid,top_words,brown_words) for (fileid, category) in test_set])
    gold = [category for (fileid, category) in test_set]

    return compute_PRF(gold, results, class_label)

def main(email_data, mails):
    random.seed(1)
    random.shuffle(email_data)
    logging.debug(str(email_data[:10]))

    # split in training, develop and test set
    train_size = int(0.8 * len(email_data))
    train_data, test_data = email_data[:train_size], email_data[train_size:]

    train_size = int(0.7 * len(train_data))
    train_data, develop_data = train_data[:train_size], train_data[train_size:]

    logging.debug("# training develop test: %s %s %s", len(train_data), len(develop_data), len(test_data))

    # limit word features to 500 top words, as training with many features takes forever
    # try different threshold on how many features work well
    for threshold in [500]:
        logging.info("\nThreshold: %d" % threshold)

        fd_train_words = nltk.FreqDist(w.lower() for w in mails.words(fileids=[f for f, c in train_data]))
        top_words = [t for t, _ in fd_train_words.most_common(threshold)]

        brown_words = set([w.lower() for w in nltk.corpus.brown.words()])

        show_most_informative = False
        results = evaluate(mails,train_data, develop_data, show_most_informative, "spam", top_words, brown_words)
        print("precision spam:", results[0])
        print("recall spam: ", results[1])
        print("f-measure spam: ", results[2])

In [6]:
# ORIGINAL SPAM MESSAGES:
# prepare review data as a list of tuples (vocab, category)
email_data = [(fileid, category) for category in mails_true.categories() for fileid in mails_true.fileids(category)]
main(email_data,mails_true)

DEBUG:root:[('nospam/8-863msg1.txt', 'nospam'), ('nospam/5-1353msg3.txt', 'nospam'), ('nospam/6-173msg2.txt', 'nospam'), ('nospam/9-1425msg2.txt', 'nospam'), ('nospam/9-148msg1.txt', 'nospam'), ('nospam/6-282msg1.txt', 'nospam'), ('nospam/3-404msg2.txt', 'nospam'), ('nospam/6-200msg1.txt', 'nospam'), ('nospam/6-4msg3.txt', 'nospam'), ('nospam/3-378msg2.txt', 'nospam')]
DEBUG:root:# training develop test: 1619 695 579
INFO:root:
Threshold: 500


precision spam: 0.8455882352941176
recall spam:  0.9745762711864406
f-measure spam:  0.905511811023622


- Report the precision, recall and f-score. (1 point)

In [39]:
mails1 = LazyCorpusLoader('mails1', CategorizedPlaintextCorpusReader,r'(?!\.).*\.txt', cat_pattern=r'(spam|nospam)/.*')

email_data1 = [(fileid, category) for category in mails1.categories() for fileid in mails1.fileids(category)]

main(email_data1,mails1)

precision spam: %s 1.0
recall spam: %s 0.9915254237288136
f-measure spam: %s 0.9957446808510638


In [51]:
mails2 = LazyCorpusLoader('mails2', CategorizedPlaintextCorpusReader,r'(?!\.).*\.txt', cat_pattern=r'(spam|nospam)/.*')

email_data2 = [(fileid, category) for category in mails2.categories() for fileid in mails2.fileids(category)]

main(email_data2, mails2)

precision spam: %s 1.0
recall spam: %s 1.0
f-measure spam: %s 1.0


- How much can you decrease
the classifier’s performance? Discuss results (1 point)

Unfornately the added features does not degrade the classifier performance. The second approache failed because it randomly included word from nltk corpus that are out of brown corpus. 